In [1]:
import os
from os import path
import pandas as pd

In [2]:
base = path.join('..', 'processed', '1-covid')

files = sorted(f for f in os.listdir(base) if f.endswith('.csv'))

files

['chaos-game-theory_r1.csv',
 'chaos-game-theory_r4.csv',
 'complex-network-k3l10.csv',
 'fourier-class-r5.csv',
 'mapping-class_r1.csv',
 'mapping-class_r2.csv',
 'shannon-entropy-k12.csv']

In [3]:
select_k_features = 500

In [4]:
%%time

index = 5

print(files[index])
frame = pd.read_csv(path.join(base,files[index]), header=None)

if frame.iloc[0,0] == "nameseq":
    frame.drop(labels=0, axis=0, inplace=True)

print(frame.shape)

frame.head()

mapping-class_r2.csv
(1717, 30820)
CPU times: user 33.5 s, sys: 731 ms, total: 34.2 s
Wall time: 34.2 s


,0,1,2,3,4,5,6,7,8,9,...,30810,30811,30812,30813,30814,30815,30816,30817,30818,30819
0,MN369046,0.75,0.50,0.75,0.25,0.5,0.75,0.50,0.75,0.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,MN306046,0.50,0.25,0.50,1.00,1.0,1.00,1.00,1.00,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,MF542265,0.75,1.00,0.50,0.25,0.5,0.50,1.00,1.00,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,KY983587,0.50,0.25,0.50,0.50,1.0,1.00,0.75,0.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,KY621348,0.50,0.50,0.75,1.00,0.5,0.75,0.25,0.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
frame.info(max_cols=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1717 entries, 0 to 1716
Columns: 30820 entries, 0 to 30819
dtypes: float64(30818), int64(1), object(1)
memory usage: 403.7+ MB


In [6]:
(30820 - 2)

30818

In [7]:
# Feature selection with ANOVA
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif

In [8]:
from collections import namedtuple
from operator import attrgetter

Feature = namedtuple('Feature', 'attr score')
get_key = attrgetter('score')

def feature_selection_anova(df, columns, target, k = 42, show=False, show_n=None):
    
    previsores = df[columns].values
    _target = df[target].values

    # Utilizando a função ANOVA
    test = SelectKBest(score_func=f_classif, k=k)
    fit = test.fit(previsores, _target)

    if show:
        if show_n is None:
            show_n = k

        scores = [ Feature(attr=name, score=score) for name, score in zip(columns, fit.scores_)]
        scores = sorted(scores, key=get_key, reverse=True)
        for i, score in enumerate(scores, start=1) :
            endline = '\t' if i % 4 else '\n'
            text = f"Feature {score.attr} : {score.score:.4f}"
            print(text.ljust(30), end=endline)
            if (show_n is not None) and i > show_n:
                break
        
    return fit

In [9]:
# Index from 0 to 30819

#     feature_selection_anova(df, columns, target, k = 42, show=False, show_n=None)
anova = feature_selection_anova(frame, range(1,30819), 30819, k = select_k_features, show=True)

Feature 29967 : 2576.4817     	Feature 29965 : 2531.1451     	Feature 29957 : 2486.4985     	Feature 29930 : 2480.9900     
Feature 29947 : 2466.9201     	Feature 29943 : 2446.6938     	Feature 29960 : 2446.0063     	Feature 29922 : 2436.7788     
Feature 29962 : 2426.6704     	Feature 29973 : 2410.6630     	Feature 29978 : 2393.9688     	Feature 29981 : 2390.4800     
Feature 29946 : 2386.7837     	Feature 29954 : 2369.3796     	Feature 29975 : 2365.8564     	Feature 29980 : 2357.3292     
Feature 29968 : 2354.9033     	Feature 29963 : 2331.8147     	Feature 29956 : 2306.4322     	Feature 29972 : 2304.9442     
Feature 29940 : 2304.7978     	Feature 29991 : 2302.4407     	Feature 29952 : 2297.3847     	Feature 30045 : 2286.0376     
Feature 29984 : 2283.6794     	Feature 29995 : 2279.7795     	Feature 29997 : 2275.5506     	Feature 29951 : 2274.1304     
Feature 29969 : 2271.2361     	Feature 29923 : 2270.5683     	Feature 29964 : 2268.5788     	Feature 29958 : 2264.1530     
Feature 

In [10]:
frame.iloc[:5, 1:30819]

,1,2,3,4,5,6,7,8,9,10,...,30809,30810,30811,30812,30813,30814,30815,30816,30817,30818
0,0.75,0.50,0.75,0.25,0.5,0.75,0.50,0.75,0.50,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.50,0.25,0.50,1.00,1.0,1.00,1.00,1.00,0.75,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.75,1.00,0.50,0.25,0.5,0.50,1.00,1.00,0.75,0.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.50,0.25,0.50,0.50,1.0,1.00,0.75,0.50,1.00,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.50,0.50,0.75,1.00,0.5,0.75,0.25,0.50,1.00,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
frame.iloc[:5,30819]

0    0
1    0
2    0
3    0
4    0
Name: 30819, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

X, X_teste, y, y_teste = train_test_split( frame.iloc[:, 1:30819],
                                           frame.iloc[:, 30819], 
                                           train_size=0.7, 
                                           test_size=0.3)


In [13]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=400, learning_rate=3.0, max_depth=1, random_state=63)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from sklearn.metrics import cohen_kappa_score, make_scorer
from imblearn.metrics import geometric_mean_score

In [15]:
%%time

pipe = Pipeline(steps=[
        ('SelectBest', SelectKBest(score_func=f_classif, k=select_k_features)),
		('MinMaxScaler', MinMaxScaler(feature_range=(0, 1))),
		('ClassificationModel', model)
        ])

scoring = {'ACC': 'accuracy', 
            'recall': 'recall', 
            'f1': 'f1', 
            'roc_auc': 'roc_auc', 
            'ACC_B': 'balanced_accuracy', 
            'kappa': make_scorer(cohen_kappa_score), 
            'gmean': make_scorer(geometric_mean_score)
        }

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(pipe, X, y, cv=kfold, scoring=scoring)

CPU times: user 26.2 s, sys: 1.72 s, total: 27.9 s
Wall time: 27.9 s


In [16]:
scores

{'fit_time': array([2.43370867, 2.40033102, 2.42511368, 2.43390465, 2.48236084,
        2.496382  , 2.39783454, 2.5015626 , 2.47707295, 2.38794017]),
 'score_time': array([0.34739017, 0.33847427, 0.33947825, 0.33899999, 0.34041786,
        0.34658241, 0.36077666, 0.36186886, 0.33741546, 0.33985162]),
 'test_ACC': array([0.9338843 , 0.8       , 0.76666667, 0.94166667, 0.73333333,
        0.94166667, 0.725     , 0.83333333, 0.85833333, 0.76666667]),
 'test_recall': array([1.        , 1.        , 1.        , 1.        , 1.        ,
        0.96226415, 1.        , 0.94252874, 1.        , 1.        ]),
 'test_f1': array([0.95833333, 0.88888889, 0.86792453, 0.96373057, 0.84615385,
        0.96682464, 0.84057971, 0.89130435, 0.91457286, 0.86792453]),
 'test_roc_auc': array([0.75787106, 0.84027778, 0.93944099, 0.80207089, 0.15625   ,
        0.9059973 , 0.65604319, 0.93312435, 0.53486169, 0.49514752]),
 'test_ACC_B': array([0.86206897, 0.5       , 0.5       , 0.87037037, 0.5       ,
        0.